In [2]:
import spacy
from spacy import displacy
import pandas as pd
from pprint import pprint
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

In [4]:
doc_file = open("/home/jovyan/work/data/text.txt")
doc_content = doc_file.read()
doc_nlp = nlp(doc_content)

In [ ]:
# removeing the stop words (most commone words that used in english and dosn't mean anything)
# check the list with nlp.

# no_stop = nlp()

# for t in doc_nlp:
#        if nlp.vocab[t.text].is_stop :

In [5]:
for t in doc_nlp:
        if t.pos_ == "PROPN" or t.pos_ == "NN":
                    print(f'{t.text:{12}} {t.pos_:{6}}')

Millfield    PROPN 
Jack         PROPN 
Thompson     PROPN 
Jack         PROPN 
Lily         PROPN 
Thompson     PROPN 
Lily         PROPN 
David        PROPN 
Williams     PROPN 
Mr.          PROPN 
Thompson     PROPN 
Lily         PROPN 
Emily        PROPN 
Davis        PROPN 
Emily        PROPN 
Ben          PROPN 
Reynolds     PROPN 
Mike         PROPN 
Smith        PROPN 
Tim          PROPN 
Johnson      PROPN 
Mayor        PROPN 
Johnson      PROPN 
Councilman   PROPN 
Smith        PROPN 
Maria        PROPN 
Hernandez    PROPN 
Jane         PROPN 
Anderson     PROPN 
Tom          PROPN 
Lee          PROPN 
Netherlands  PROPN 
Opa          PROPN 
Pieter       PROPN 
Pieter       PROPN 
Jaap         PROPN 
Jaap         PROPN 
Jaap         PROPN 
Annelies     PROPN 
Jan          PROPN 
Jan          PROPN 
Jaap         PROPN 
Karel        PROPN 
Mieke        PROPN 
Jaap         PROPN 
Jaap         PROPN 
Jaap         PROPN 
Netherlands  PROPN 
Netherlands  PROPN 
Pieter       PROPN 


In [6]:
for ent in doc_nlp.ents:
  if ent.label_ == "PERSON":
    print(f'{ent.text:{25}} {ent.label_:{6}} {ent.start_char:{6}} {ent.end_char:{6}}')

Millfield                 PERSON    136    145
Jack Thompson             PERSON    171    184
Jack                      PERSON    186    190
Lily Thompson             PERSON    229    242
David Williams            PERSON    282    296
Thompson                  PERSON    350    358
Lily                      PERSON    370    374
Emily Davis               PERSON    389    400
Ben Reynolds              PERSON    475    487
Mike Smith                PERSON    535    545
Tim Johnson               PERSON    550    561
Johnson                   PERSON    725    732
Councilman Smith          PERSON    737    753
Maria Hernandez           PERSON    810    825
Jane Anderson             PERSON    978    991
Tom Lee                   PERSON   1007   1014
Opa                       PERSON   1213   1216
Pieter                    PERSON   1237   1243
Pieter                    PERSON   1296   1302
Jaap                      PERSON   1377   1381
Jaap                      PERSON   1383   1387
Jaap         

In [7]:
pattern1 = [{"ENT_TYPE": "PERSON"}]
pattern2 = [{"POS": "PROPN"},{"POS": "PROPN"}]
pattern3 = [{"POS": "NN"},{"POS": "NN"}]
pattern4 = [{"POS": "PROPN"},{"POS": "NN"}]
pattern5 = [{"POS": "NN"},{"POS": "PROPN"}]
pattern6 = [{"POS": "NN"}]
pattern7 = [{"POS": "PROPN"}]
pattern8 = [{"POS": "PROPN"},{"POS": "PROPN"},{"POS": "PROPN"}]
pattern9 = [{"POS": "NN"},{"POS": "NN"},{"POS": "NN"}]
pattern10 = [{"POS": "PROPN"},{"POS": "PROPN"},{"POS": "NN"}]
pattern11 = [{"POS": "PROPN"},{"POS": "NN"},{"POS": "NN"}]

matcher.add("Names",[pattern1,pattern2,pattern3,pattern4,pattern5,pattern6,pattern7,pattern8,pattern9,pattern10,pattern11])
matches = matcher(doc_nlp)

In [8]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc_nlp[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

10631222085860127603 Names 30 31 Millfield
10631222085860127603 Names 37 38 Jack
10631222085860127603 Names 37 39 Jack Thompson
10631222085860127603 Names 38 39 Thompson
10631222085860127603 Names 40 41 Jack
10631222085860127603 Names 49 50 Lily
10631222085860127603 Names 49 51 Lily Thompson
10631222085860127603 Names 50 51 Thompson
10631222085860127603 Names 52 53 Lily
10631222085860127603 Names 60 61 David
10631222085860127603 Names 60 62 David Williams
10631222085860127603 Names 61 62 Williams
10631222085860127603 Names 72 73 Mr.
10631222085860127603 Names 72 74 Mr. Thompson
10631222085860127603 Names 73 74 Thompson
10631222085860127603 Names 79 80 Lily
10631222085860127603 Names 83 84 Emily
10631222085860127603 Names 83 85 Emily Davis
10631222085860127603 Names 84 85 Davis
10631222085860127603 Names 93 94 Emily
10631222085860127603 Names 101 102 Ben
10631222085860127603 Names 101 103 Ben Reynolds
10631222085860127603 Names 102 103 Reynolds
10631222085860127603 Names 114 115 Mike
10